In [ ]:
#| hide
#| default_exp util

# util

> Small utility functions for the `ouscope` library.

In [ ]:
#| export
from __future__ import annotations

In [ ]:
#| hide
from nbdev.showdoc import *
from nbdev import *
from fastcore.basics import patch

In [ ]:
#| exporti
import os
from fastcore.basics import patch
from ouscope.core import Telescope
from tqdm.auto import tqdm

In [ ]:
#| exporti
def print_dict(d):
    for k, v in d.items():
        print(f'{k}: {v}')

In [ ]:
#| login
scope=Telescope(config='~/.config/telescope.ini')

In [ ]:
#| export
@patch
def get_object_obs(self: Telescope, obj: str):
    '''
    Find all jobs for a given object.
    '''
    reqlst=self.get_user_requests(sort='completion')
    print(f'Number of users requests: {len(reqlst)}')

    complete = [rq for rq in sorted(reqlst, key=lambda r: int(r['requesttime']), reverse=True) 
                        if Telescope.REQUESTSTATUS_TEXTS[int(rq['status'])]=='Complete']
    print('Completed:', len(complete))
    objjobs = ((int(self.get_request(int(j['id']))['jid']), int(j['id'])) 
               for j in complete if j['objectname']==obj)
    return objjobs

In [ ]:
#| login
reqlst=scope.get_user_requests(sort='completion')
complete = [rq for rq in sorted(reqlst, key=lambda r: int(r['requesttime']), reverse=True) 
                        if Telescope.REQUESTSTATUS_TEXTS[int(rq['status'])]=='Complete']

In [ ]:
#| login

# Get all observations of VS
VS = 'BI Her'
jlist = sorted(list(scope.get_object_obs(VS)))
print(f'Completed observations of {VS}: {len(jlist)}')

for jid, rid in tqdm(jlist):
    # print(f"R{rid}:J{jid}")
    try :
        data = scope.get_obs(scope.get_job(jid), cube=True, verbose=False)
        if data :
            fp = f'VS/{"_".join(VS.split())}'
            os.makedirs(fp, exist_ok=True)
            fn = f'{jid}'
            try :
                # print(os.path.abspath(data.name), f'{fp}/{fn}.fits')
                os.symlink(os.path.abspath(data.name), f'{fp}/{fn}.fits')
            except FileExistsError:
                pass
            # with open(f'/home/jochym/Astro/VS/{"_".join(VS.split())}/{jid}.fits', 'wb') as ff:
            #     ff.write(data.read())
        else :
            print(f'Download of J{jid} failed (no data)')
    except TimeoutError:
        print(f'Download of J{jid} failed (timeout)')
        continue

Number of users requests: 1700
Completed: 1679
Completed observations of BI Her: 173


  0%|          | 0/173 [00:00<?, ?it/s]

In [ ]:
#| login

# get all completed jobs
for rq in (pbar := tqdm(complete[:25])):
    name = rq['objectname']
    rid = int(rq['id'])
    pbar.set_postfix_str(name)
    try:
        jid = int(scope.get_request(rid)['jid'])
        data = scope.get_obs(scope.get_job(jid), cube=True, verbose=False)
        if data :
            fp = f'VS/{"_".join(name.split())}'
            os.makedirs(fp, exist_ok=True)
            fn = f'{jid}'
            try :
                os.symlink(os.path.abspath(data.name), f'{fp}/{fn}.fits')
            except FileExistsError:
                pass
        else :
            print(f'Download of J{jid} failed')
    except TimeoutError :
        print(f'Download of J{jid} failed, timeout')
        pass

  0%|          | 0/25 [00:00<?, ?it/s]